# LABELLING - ACTIVE LEARNING

In [15]:
%pip install transformers datasets evaluate

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
import pandas as pd
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer
from datasets import Dataset, DatasetDict
import evaluate
import numpy as np
import json

## 1. Labelling and Finetuning functions

In [11]:
'''
Function that labels the data with the provided model
and saves the labeled data to a csv file. Additionally,
it saves 100 rows with the lowest RoBERTa confidence scores
to a new CSV file.

Params:
model - the model to be used for sentiment analysis
tokenizer - the tokenizer to be used for sentiment analysis
dataset - dataframe containing the entire dataset
round - active learning round
'''
def label_data(model, tokenizer, dataset, round):
  # Initialize the sentiment analysis pipeline
  sentiment_pipeline = pipeline("text-classification", 
                                model=model,
                                tokenizer=tokenizer,
                                device=0) 
  
  # Extract the text column of selected_data as a list
  reviews = dataset["text"].tolist()
    
  # Calculate the sentiment of the each of the reviews
  print(f"\nRound {round} - Automated Labelling ")
  print("Predicting sentiment labels of data...")

  kwargs = {'padding':True,'truncation':True,'max_length':512}
  results = sentiment_pipeline(reviews, **kwargs) 

  print("Sentiment labels predicted.")
  print("Saving labeled data to a csv files...")

  # Add the sentiment and score to the selected_data DataFrame
  label2id = {"positive": 1, "negative": -1, "neutral": 0}
  dataset["roberta_label"] = [label2id[res["label"]] for res in results]
  dataset["roberta_score"] = [res["score"] for res in results]

  # Save the labeled data to a csv file
  dataset.to_csv(f'../Data/Labelling/round{round}_roberta_labelled_all_data.csv', index=False)

  # Save 100 rows with the lowest RoBERTa confidence scores to a new CSV file
  df_low_confidence = dataset.nsmallest(100, 'roberta_score')
  df_low_confidence.to_csv(f'../Data/Labelling/round{round}_roberta_labelled_low_confidence.csv', index=False)
  
  print(f"Completed Round {round} - Automated Labeling")

  return dataset

In [ ]:
# Load, process and tokenize the manual train and eval data for each round
def process_manual_data(tokenizer, round):
  
  # Load the manual data for the round, and all the rounds before it (to retain previously learnt patterns)
  train_data = pd.read_csv(f'../Data/Labelling/Manual/round{round}_manual_low_confidence.csv')
  for i in range(1, round):
    round_data = pd.read_csv(f'../Data/Labelling/Manual/round{i}_manual_low_confidence.csv')
    train_data = pd.concat([train_data, round_data], ignore_index=True)

  val_data = pd.read_csv(f'../Data/Labelling/Manual/manual_val_set.csv')

  # Drop all columns except the text and the manual label
  train_data.drop(columns=[col for col in train_data.columns if col not in ['text', 'manual_label']], inplace=True)
  val_data.drop(columns=[col for col in val_data.columns if col not in ['text', 'manual_label']], inplace=True)

  # Drop the duplicates from the training manual data, keeping the first occurence (latest label)
  train_data.drop_duplicates(subset=['text'], keep='first', inplace=True)

  # Drop the rows which are NaN, or contain '2' values in the manual_label column (rows marked irrelevant during manual labelling)
  train_data = train_data[train_data['manual_label'] != 2].dropna()

  # Rename the manual_label column to labels
  train_data.rename(columns={'manual_label':'labels'}, inplace=True)
  val_data.rename(columns={'manual_label':'labels'}, inplace=True)

  # Map the labels to the model's labels
  # 0 -> Negative, 1 -> Neutral, 2, Positive

  label2id = {-1: 0, 0: 1, 1: 2}
  train_data['labels'] = train_data['labels'].map(label2id)
  val_data['labels'] = val_data['labels'].map(label2id)

  # Print the number of rows in the training and validation data
  print(f"Round {round} - Training data: {train_data.shape[0]} rows")
  print(f"Round {round} - Validation data: {val_data.shape[0]} rows")
  
  # Convert to Dataset object
  train_data = Dataset.from_pandas(train_data)
  val_data = Dataset.from_pandas(val_data)
  
  # Tokenize the data using the model's tokenizer
  train_data_tokenized = train_data.map(
    lambda instance: tokenizer(instance["text"], truncation=True, max_length=512),
    batched=True
  )

  val_data_tokenized = val_data.map(
    lambda instance: tokenizer(instance["text"], truncation=True, max_length=512),
    batched=True
  )

  # Combine the train and val datasets
  train_val_dataset = DatasetDict({"train": train_data_tokenized, "val": val_data_tokenized})

  print(f"Round {round} - Manual data loaded and processed.")

  return train_val_dataset
    

In [ ]:
# Function to compute the metrics for the model
def compute_metrics(eval_preds):
    # Load the metrics - accuracy and f1 score
    accuracy = evaluate.load("accuracy")
    f1 = evaluate.load("f1")
    precision = evaluate.load("precision")
    recall = evaluate.load("recall")


    # Extract the logits and true labels from the predictions
    logits, true_labels = eval_preds

    # Compute the predicted label (label with the highest logit score)
    pred_labels = np.argmax(logits, axis=-1)

    # Compute the metrics using the true and predicted labels
    computed_accuracy = accuracy.compute(predictions=pred_labels, 
                                       references=true_labels)
    computed_f1 = f1.compute(predictions=pred_labels,
                                references=true_labels)
    computed_precision = precision.compute(predictions=pred_labels,
                                references=true_labels)
    computed_recall = recall.compute(predictions=pred_labels,
                                references=true_labels)
    
    # Return the computed metrics
    computed_metrics = {
        "accuracy": computed_accuracy,
        "f1": computed_f1,
        "precision": computed_precision,
        "recall": computed_recall
    }
                                
    return computed_metrics

In [ ]:
# Reference: https://huggingface.co/docs/transformers/en/training
def finetune(model, dataset, tokenizer, round):
  # Define a data collator object for dynamic padding (padding to the maximum length of the batch)
  data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

  # Configure the training arguments
  training_arguments = TrainingArguments(
    output_dir = f'../Models/round{round}_finetuned_model_checkpoints/',
    num_train_epochs = 3,
    eval_strategy="epoch",
    )
  
  trainer = Trainer(
    model,
    training_arguments,
    train_dataset = dataset['train'],
    eval_dataset = dataset['val'],
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics=compute_metrics,
  )

  print(f"\nRound {round} - Fine-tuning the model...")

  trainer.train()

  # Calculate the metrics on the validation set
  eval_results = trainer.evaluate()
  print(f"Round {round} - Evaluation results: {eval_results}")

  # Save evaluation results to a json file
  with open(f'../Models/Evaluation/round{round}_finetuned_model_eval_results.json', 'w') as f:
    json.dump(eval_results, f, indent=4)
    

  trainer.save_model(f'../Models/round{round}_finetuned_model')
  print(f"Round {round} - Model fine-tuned.")

  # Return the finetuned model
  return trainer.model

In [ ]:
# Function to conduct one round of active learning
def active_learning(pretrained_model, dataset, tokenizer, round):
  print (f"Round {round} of Active Learning")

  # 1. Load the model
  if round == 1:
    # For round 1, load the pretrained model
    model = AutoModelForSequenceClassification.from_pretrained(pretrained_model)
  else:
    # For subsequent rounds, load the finetuned model from the previous round
    model = AutoModelForSequenceClassification.from_pretrained(f'../Models/round{round-1}_finetuned_model')
      
  # 2. Using the model, automatically label the entire dataset
  label_data(model = model, 
            dataset = dataset, 
            tokenizer = tokenizer,
            round = round)
  
  # 3. Load the manually labeled data, including the newly labeled data from the previous round
  train_val_data = process_manual_data(tokenizer = tokenizer,
                                    round = round) 
  
  # 4. Fine-tune the model on the manually labeled data
  finetune(model = model, 
          dataset = train_val_data,
          tokenizer = tokenizer,
          round = round)
      
  print (f"Completed Round {round} of Active Learning")


## 2. Run Active Learning Loop

Active learning allows us to manually label the most informative parts of the dataset that confuses the model the most. 

In [ ]:
# Load the selected data, and tokenizer
pretrained_model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
dataset = pd.read_csv('../Data/selected_data.csv')
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)

In [ ]:
# Conduct active learning - round 1
active_learning(pretrained_model = pretrained_model_name, 
                tokenizer = tokenizer, 
                dataset = dataset, 
                round = 1)

In [ ]:
# Conduct active learning - round 2
active_learning(pretrained_model = pretrained_model_name, 
                tokenizer = tokenizer, 
                dataset = dataset, 
                round = 2)

In [ ]:
# Conduct active learning - round 3
active_learning(pretrained_model = pretrained_model_name, 
                tokenizer = tokenizer, 
                dataset = dataset, 
                round = 3)

In [ ]:
# Conduct active learning - round 4
active_learning(pretrained_model = pretrained_model_name, 
                tokenizer = tokenizer, 
                dataset = dataset, 
                round = 4)

In [ ]:
# Conduct active learning - round 5
active_learning(pretrained_model = pretrained_model_name, 
                tokenizer = tokenizer, 
                dataset = dataset, 
                round = 5)